In [1]:
from mec_env_var import *
from helper import *
import tensorflow as tf
import ipdb as pdb
import time

for k in range(10):
    
    tf.reset_default_graph()
    
    print('---------' + str(k) + '------------')
    
    MAX_EPISODE = 100
    MAX_EPISODE_LEN = 10000

    NUM_T = 1
    NUM_R = 4
    SIGMA2 = 1e-9

    t_factor = 0.8
    noise_sigma = 0.12

    config = {'state_dim':3, 'action_dim':2};
    train_config = {'minibatch_size':64, 'actor_lr':0.0001, 'tau':0.001, 
                    'critic_lr':0.001, 'gamma':0.99, 'buffer_size':250000, 
                    'random_seed':int(time.clock()*1000%1000), 'noise_sigma':noise_sigma, 'sigma2':SIGMA2}
    
#     rate = 3.0
    res_path = 't_M_08_nB/'
    model_path = 'model_M_08_nB/my_train_model_'+str(k)+'-2000'
    meta_path = model_path+'.meta'    
    user_config = [{'id':'1', 'model':'AR', 'num_r':NUM_R, 'rate':1.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'2', 'model':'AR', 'num_r':NUM_R, 'rate':2.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'3', 'model':'AR', 'num_r':NUM_R, 'rate':3.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000}]
    
    print(user_config)
    # 0. initialize the session object
    sess = tf.Session() 

    # gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
    # sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

    # 1. include all user in the system according to the user_config
    user_list = [];
    for info in user_config:
        info.update(config)
        info['model_path'] = model_path
        info['meta_path'] = info['model_path']+'.meta'
        user_list.append(MecTermLD(sess, info, train_config))
        print('Initialization OK!----> user ' + info['id'])

    # 2. create the simulation env
    env = MecSvrEnv(user_list, NUM_R, SIGMA2, MAX_EPISODE_LEN)

#     sess.run(tf.global_variables_initializer())

#     env.init_target_network()

    # #Create a saver object which will save all the variables
    # saver = tf.train.Saver() 

    res_r = []
    res_p = []
    res_b = []
    res_o = []
    res_d = []
    # 3. start to explore for each episode
    for i in range(MAX_EPISODE):

        cur_init_ds_ep = env.reset(isTrain=False)
        
        cur_r_ep = np.zeros(len(user_list))
        cur_p_ep = np.zeros(len(user_list))
        cur_op_ep = np.zeros(len(user_list))
        cur_ts_ep = np.zeros(len(user_list))
        cur_ps_ep = np.zeros(len(user_list))
        cur_rs_ep = np.zeros(len(user_list))
        cur_ds_ep = np.zeros(len(user_list))
        cur_ch_ep = np.zeros(len(user_list))
        cur_of_ep = np.zeros(len(user_list))

        for j in range(MAX_EPISODE_LEN):

            # first try to transmit from current state
            [cur_r, done, cur_p, cur_op, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit()

            cur_r_ep += cur_r
            cur_p_ep += cur_p
            cur_op_ep += cur_op
            cur_ts_ep += cur_ts
            cur_ps_ep += cur_ps
            cur_rs_ep += cur_rs
            cur_ds_ep += cur_ds
            cur_ch_ep += cur_ch
            cur_of_ep += cur_of

            if done:
                res_r.append(cur_r_ep/MAX_EPISODE_LEN)
                res_p.append(cur_p_ep/MAX_EPISODE_LEN)
                res_b.append(cur_ds_ep/MAX_EPISODE_LEN)
                res_o.append(cur_of_ep/MAX_EPISODE_LEN)
                res_d.append(cur_ds)
                print('%d:r:%s,p:%s,op:%s,tr:%s,pr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s,rbuf:%s' % (i, cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_op_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_ps_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep, cur_ds))

    name = res_path+'test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
    np.savez(name, res_r, res_p, res_b, res_o, res_d)
    
    sess.close()


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
---------0------------
[{'id': '1', 'model': 'AR', 'num_r': 4, 'rate': 1.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.8, 'penalty': 1000}, {'id': '2', 'model': 'AR', 'num_r': 4, 'rate': 2.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.8, 'penalty': 1000}, {'id': '3', 'model': 'AR', 'num_r': 4, 'rate': 3.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.8, 'penalty': 1000}]
INFO:tensorflow:Restoring parameters from model_M_08_nB/my_train_model_0-2000
Initialization OK!----> user 1
INFO:tensorflow:Restoring parameters from model_M_08_nB/my_train_model_0-2000
Initialization OK!----> user 2
INFO:tensorflow:Restoring parameters from model_M_08_nB/my_train_model_0-2000
Initialization OK!----> user 3
0:r:[ -1.91380982  -7.28972201 -22.99348226],p:[0.15947552 0.65638751 2.26351101],op:[1.62011486e-03 6.03250310e-05 0.00000000e+0

20:r:[ -1.87044296  -7.09925913 -22.77852404],p:[0.15482517 0.6425887  2.23903304],op:[1.89172193e-03 5.91652263e-05 0.00000000e+00],tr:[0.         0.7324747  0.72358715],pr:[1.01484197 1.25813972 2.2642832 ],rev:[0.9826 1.9902 2.988 ],dbuf:[ 3.08353928  9.79038125 24.33129864],ch:[4.02245705 4.19177877 3.97230014],ibuf:[0, 0, 0],rbuf:[ 1.32739537 16.83823245 28.26404364]
21:r:[ -1.83958193  -7.20896031 -22.99812774],p:[0.15343215 0.65220284 2.26520828],op:[1.75310680e-03 3.57055455e-05 0.00000000e+00],tr:[0.         0.703984   0.71951843],pr:[1.01441833 1.28219266 2.26967641],rev:[0.9843 1.9848 2.9887],dbuf:[ 2.99049934  9.95525957 24.38230757],ch:[3.85933852 4.01434588 3.92140634],ibuf:[0, 0, 0],rbuf:[ 5.5055334   4.38057013 23.31564938]
22:r:[ -1.85307157  -7.47702846 -22.72951316],p:[0.15432662 0.67048063 2.23549052],op:[1.68479607e-03 3.57256508e-05 0.00000000e+00],tr:[0.         0.71802021 0.74688715],pr:[1.01876101 1.2928732  2.25432045],rev:[0.9896 2.0107 3.0011],dbuf:[ 3.02490

42:r:[ -1.9041329   -7.12592236 -22.82326418],p:[0.15842325 0.64701774 2.24536615],op:[1.63869300e-03 3.64881179e-05 0.00000000e+00],tr:[0.         0.73310535 0.69570311],pr:[1.02611308 1.2689628  2.28342987],rev:[0.9979 2.0009 2.9787],dbuf:[ 3.11818691  9.74744266 24.3016748 ],ch:[3.85874057 4.08004949 3.8755461 ],ibuf:[0, 0, 0],rbuf:[ 4.62398047  4.28161168 23.67801358]
43:r:[ -1.88315903  -7.27298708 -22.99344059],p:[0.15574355 0.6568053  2.26598197],op:[1.76433734e-03 4.10466058e-05 0.00000000e+00],tr:[0.         0.70512003 0.7297242 ],pr:[1.02178557 1.28234764 2.27313979],rev:[0.992  1.9873 3.0026],dbuf:[ 3.11547967 10.0910817  24.32792397],ch:[4.02370981 4.08102935 3.92707315],ibuf:[0, 0, 0],rbuf:[ 3.35397333  4.17075863 21.03811499]
44:r:[ -1.87609618  -7.00636441 -22.64330815],p:[0.15603177 0.6387775  2.22317691],op:[1.70317296e-03 3.74593695e-05 0.00000000e+00],tr:[0.         0.73199563 0.71754514],pr:[1.02145143 1.26272558 2.26447537],rev:[0.9915 1.9958 2.9822],dbuf:[ 3.07108

64:r:[ -1.87917165  -7.36182317 -22.97731597],p:[0.15543277 0.65349187 2.26271422],op:[1.90973427e-03 2.09071148e-05 0.00000000e+00],tr:[0.         0.68630061 0.73036973],pr:[1.01871046 1.28737006 2.26981192],rev:[0.9861 1.974  3.0004],dbuf:[ 3.1021582  10.66860468 24.37801093],ch:[3.95513382 3.88225743 3.94938251],ibuf:[0, 0, 0],rbuf:[ 7.15916807  6.53972113 23.82905638]
65:r:[ -1.92917487  -7.40752262 -22.68695159],p:[0.16002052 0.66220329 2.23272633],op:[1.67216681e-03 3.27765168e-05 0.00000000e+00],tr:[0.         0.71920908 0.7672536 ],pr:[1.03069423 1.28531282 2.23468166],rev:[1.0014 2.0053 3.0017],dbuf:[ 3.17816688 10.54817038 24.12570465],ch:[3.90446521 4.01968471 4.17851556],ibuf:[0, 0, 0],rbuf:[ 2.93204122 15.64279422 21.47637312]
66:r:[ -1.90786088  -7.55920455 -22.84266082],p:[0.15952069 0.67651433 2.24655923],op:[1.59419820e-03 2.74934991e-05 0.00000000e+00],tr:[0.         0.72179411 0.7205574 ],pr:[1.03293307 1.2986799  2.273275  ],rev:[1.0061 2.0206 2.9941],dbuf:[ 3.09470

86:r:[ -1.91344835  -7.51249825 -22.70365899],p:[0.15781144 0.67013285 2.23316209],op:[1.79062576e-03 2.41047891e-05 0.00000000e+00],tr:[0.         0.70212081 0.72762098],pr:[1.02422687 1.29945612 2.25499945],rev:[0.994  2.0012 2.9825],dbuf:[ 3.18315903 10.75621287 24.19181148],ch:[4.0614365  3.95043666 4.03410287],ibuf:[0, 0, 0],rbuf:[ 1.36197227 11.8260999  22.70745954]
87:r:[ -1.9347351   -7.310118   -22.89035834],p:[0.16157914 0.65725489 2.25247392],op:[1.68534895e-03 2.36497777e-05 0.00000000e+00],tr:[0.         0.70486834 0.73824809],pr:[1.03498398 1.28515593 2.26288975],rev:[1.0061 1.9906 3.0012],dbuf:[ 3.14309585 10.25944833 24.3528349 ],ch:[3.88243059 4.04682224 4.09588061],ibuf:[0, 0, 0],rbuf:[ 2.44330274 18.65107679 23.3290637 ]
88:r:[ -1.91032317  -7.50172645 -22.95652738],p:[0.15934297 0.67076775 2.26247242],op:[1.59857115e-03 1.97515869e-05 0.00000000e+00],tr:[0.         0.71104533 0.72571408],pr:[1.02930928 1.29842803 2.27210433],rev:[1.0021 2.0078 2.9976],dbuf:[ 3.11395

...

15:r:[-2.17400820e+00 -7.27495958e+00 -9.32622101e+04],p:[0.17841718 0.68999262 0.        ],op:[1.58470154e-07 5.52968927e-05 0.00000000e+00],tr:[0.35255483 0.7313939  0.        ],pr:[0.63699778 1.27199241 0.        ],rev:[0.9896 2.0029 2.988 ],dbuf:[3.73334733e+00 8.77288126e+00 4.66311051e+05],ch:[4.08824575 3.96772023 3.94833851],ibuf:[0, 0, 0],rbuf:[4.90048796e+00 8.93283431e+00 4.81211000e+05]
16:r:[-2.25689873e+00 -7.15412187e+00 -9.92238378e+04],p:[0.18390958 0.6792612  0.        ],op:[1.63346128e-06 9.51508809e-05 0.00000000e+00],tr:[0.36137005 0.72457565 0.        ],pr:[0.64101995 1.2708308  0.        ],rev:[1.0022 1.9955 2.9967],dbuf:[3.92804529e+00 8.59635515e+00 4.96119189e+05],ch:[4.08541645 3.99188397 4.11284575],ibuf:[0, 0, 0],rbuf:[3.92401979e+00 1.57307519e+01 5.11178000e+05]
17:r:[-2.32017351e+00 -7.46673843e+00 -1.05223434e+05],p:[0.18701249 0.70055126 0.        ],op:[0.00000000e+00 7.60272562e-05 0.00000000e+00],tr:[0.35906337 0.73541319 0.        ],pr:[0.6593538  1

36:r:[-2.29564814e+00 -7.34819534e+00 -2.19143666e+05],p:[0.18704247 0.69674701 0.        ],op:[3.30561315e-07 1.71958270e-04 0.00000000e+00],tr:[0.3691627  0.74427032 0.        ],pr:[0.64373114 1.27311002 0.        ],rev:[1.0126 2.0173 3.0005],dbuf:[3.99652859e+00 8.86421800e+00 1.09571833e+06],ch:[4.0229888  3.95096216 3.92574152],ibuf:[0, 0, 0],rbuf:[3.93746978e+00 1.62686162e+01 1.11071900e+06]
37:r:[-2.33813596e+00 -7.37626083e+00 -2.25086792e+05],p:[0.18699074 0.69375554 0.        ],op:[1.11741808e-07 1.32722978e-04 0.00000000e+00],tr:[0.35577327 0.74117075 0.        ],pr:[0.65666438 1.27117768 0.        ],rev:[1.0127 2.0113 2.9522],dbuf:[4.21104591e+00 9.12577363e+00 1.12543396e+06],ch:[4.02322375 4.0040374  4.00343163],ibuf:[0, 0, 0],rbuf:[6.59196237e+00 1.37215702e+01 1.14024100e+06]
38:r:[-2.30909515e+00 -7.15336223e+00 -2.31033934e+05],p:[0.18540467 0.67803913 0.        ],op:[1.32423451e-06 1.27319667e-04 0.00000000e+00],tr:[0.3478518  0.71969031 0.        ],pr:[0.65953463 1

57:r:[-2.26976919e+00 -7.31480890e+00 -3.44964475e+05],p:[0.18215048 0.68469914 0.        ],op:[2.14104056e-06 1.11322459e-04 0.00000000e+00],tr:[0.34234075 0.72037849 0.        ],pr:[0.65729019 1.2683323  0.        ],rev:[0.9989 1.9884 2.9833],dbuf:[4.06274118e+00 9.18162608e+00 1.72482237e+06],ch:[3.92129686 3.99436062 3.94657135],ibuf:[0, 0, 0],rbuf:[4.32717978e+00 1.09626860e+01 1.73976800e+06]
58:r:[-2.28798212e+00 -7.41902207e+00 -3.50955783e+05],p:[0.18421606 0.69805727 0.        ],op:[4.74247803e-08 1.08007488e-04 0.00000000e+00],tr:[0.34200702 0.74217175 0.        ],pr:[0.66194522 1.2722493  0.        ],rev:[1.0036 2.0133 3.004 ],dbuf:[4.07126647e+00 9.16849939e+00 1.75477892e+06],ch:[3.89091707 4.06673911 4.08674729],ibuf:[0, 0, 0],rbuf:[8.28556309e-01 5.99866354e+00 1.76980800e+06]
59:r:[-2.10991822e+00 -7.40829534e+00 -3.56980404e+05],p:[0.17130474 0.69492576 0.        ],op:[1.20473615e-06 7.04357999e-05 0.00000000e+00],tr:[0.32855155 0.74893402 0.        ],pr:[0.64120892 1

78:r:[-2.19653870e+00 -7.14336381e+00 -4.70945068e+05],p:[0.17748057 0.67442766 0.        ],op:[0.         0.00012855 0.        ],tr:[0.34509451 0.7130316  0.        ],pr:[0.63916406 1.268852   0.        ],rev:[0.9845 1.9816 2.9822],dbuf:[3.88347059e+00 8.73457067e+00 2.35472534e+06],ch:[4.01444981 3.94453856 4.02708356],ibuf:[0, 0, 0],rbuf:[5.59420394e+00 5.45040627e+00 2.36959200e+06]
79:r:[-2.30829212e+00 -7.36196247e+00 -4.76896106e+05],p:[0.18488066 0.69445125 0.        ],op:[7.95435173e-08 1.07591416e-04 0.00000000e+00],tr:[0.34658453 0.73354104 0.        ],pr:[0.65880367 1.27102736 0.        ],rev:[1.005  2.0037 2.9676],dbuf:[4.14623114e+00 9.02745871e+00 2.38448053e+06],ch:[3.96471819 4.01174711 3.96785172],ibuf:[0, 0, 0],rbuf:[1.79913037e+00 3.69197728e+00 2.39926800e+06]
80:r:[-2.23820509e+00 -7.41887965e+00 -4.82812907e+05],p:[0.18508897 0.69534496 0.        ],op:[1.04439930e-07 9.55920786e-05 0.00000000e+00],tr:[0.3712704  0.72186455 0.        ],pr:[0.63519893 1.27329513 0.

99:r:[-2.23990058e+00 -7.02450720e+00 -5.96738513e+05],p:[0.18210685 0.6683836  0.        ],op:[3.81744735e-07 1.15978550e-04 0.00000000e+00],tr:[0.35412509 0.72623981 0.        ],pr:[0.64200908 1.26513252 0.        ],rev:[0.9969 1.991  3.0056],dbuf:[3.91521360e+00 8.38255304e+00 2.98369256e+06],ch:[4.0004812  4.09169251 4.00218109],ibuf:[0, 0, 0],rbuf:[8.67172663e+00 4.66470859e+00 2.99868000e+06]


In [2]:
name = res_path+'/test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
# print(name)
np.savez(name, res_r, res_p, res_b, res_o, res_d)

In [ ]:
graph = tf.get_default_graph()
# variable_names = [v.name for v in tf.trainable_variables()]
# values = sess.run(variable_names)
# tf.trainable_variables()

inputs = graph.get_tensor_by_name("input_1/X"+":0")
scaled_out = graph.get_tensor_by_name("output_1"+":0")
print sess.run(scaled_out, feed_dict={inputs: [[7.22254768,3.9729721,0.7729721]]})
#Create a saver object which will save all the variables
saver = tf.train.Saver()
 
#Now, save the graph
saver.save(sess, 'model/my_train_model',global_step=2000)

# user_list[0].agent.actor.predict([[7.22254768,3.9729721,0.7729721]]) # [[0.63916105, 0.20230901]]